# Start here if working in Colab

In [0]:
# Mount Google Drive
%%capture
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive

drive.mount(ROOT);           # we mount the google drive at /content/drive

In [0]:
# Set working directory
%%capture
%cd /content/drive/My Drive/restoration-mapper

# Start here if working locally

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
!pip install -U keras-tuner
from kerastuner.tuners import RandomSearch

### Import Data

In [0]:
test_x = np.load('data/test-processed/test_x_l2a_processed.npy')
test_y = np.load('data/test-processed/test_y_l2a_processed.npy')
test_length = np.load('data/test-processed/test_length_l2a_processed.npy')

train_x = np.load('data/train-processed/data_x_l2a_processed.npy')
train_y = np.load('data/train-processed/data_y_l2a_processed.npy')
train_length = np.load('data/train-processed/length_l2a_processed.npy')

In [0]:
# Remove unknown x variable
train_x = np.delete(train_x, 14, -1)
test_x = np.delete(test_x, 14, -1)

### Normalize data

In [0]:
# Normalize data [x_norm = (x - min) / (max - min)]
train_x_min = np.min(train_x, axis = (0,1,2,3))
train_x_range = np.max(train_x, 
                       axis = (0,1,2,3)) - np.min(train_x, axis = (0,1,2,3))

def np_normalize(array, x_min, x_range):
  n = np.prod(array.shape[:-1])
  dim = array.shape
  return (array - np.tile(x_min, 
                          n).reshape(dim)) / np.tile(x_range, n).reshape(dim)

train_x_norm = np_normalize(train_x, train_x_min, train_x_range)
test_x_norm = np_normalize(test_x, train_x_min, train_x_range)

In [0]:
# Check if data is normalized
print("Min value of all variables is 0:", 
      (np.min(train_x_norm[:,:,:,:,:], axis=(0,1,2,3)) == 0).all())
print("Max value of all variables is 1:", 
      (np.max(train_x_norm[:,:,:,:,:], axis=(0,1,2,3)) == 1).all())

In [0]:
# Subset data to first time period
train_x_norm = train_x_norm[:, 0, :, :, :]
test_x_norm = test_x_norm[:, 0, :, :, :]

### Tune Model

See documentation: https://keras-team.github.io/keras-tuner/

In [0]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=hp.Int('layer1_filters',
                                        min_value=16,
                                        max_value=160,
                                        step=16),
                                  kernel_size = (3, 3),
                                  strides = 1,
                                  padding = "same",
                                  activation='relu',
                                  input_shape = (16, 16, 16),
                                  kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2', values=[0.5, 0.05, 0.05])),
                                  kernel_initializer='glorot_uniform'))
    for i in range(hp.Int('num_otherlayers', 1, 5)):
      model.add(keras.layers.Conv2D(filters=hp.Int('otherlayer_filters',
                                                 min_value=16,
                                                 max_value=320,
                                                 step=16), 
                                    kernel_size = (3, 3), 
                                    strides = 1, 
                                    padding = "same", 
                                    activation='relu',
                                    kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2', values=[0.5, 0.05, 0.05])),
                                    kernel_initializer='glorot_uniform'))
    model.add(keras.layers.Conv2D(filters=1, 
                                  kernel_size = (3, 3), 
                                  strides = 1, 
                                  padding = "valid", 
                                  activation='sigmoid',
                                  kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2', values=[0.5, 0.05, 0.05])),
                                  kernel_initializer='glorot_uniform'))
    model.add(keras.layers.Reshape((14,14)))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4, 1e-5])),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'])
    return model

In [0]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3)

tuner.search_space_summary()

In [0]:
tuner.search(train_x_norm, train_y,
             epochs=15,
             validation_data=(test_x_norm, test_y))

In [0]:
models = tuner.get_best_models(num_models=5)
tuner.results_summary()